In [1]:
from typing import Optional
from smolagents import HfApiModel, LiteLLMModel, TransformersModel, tool,OpenAIServerModel
from smolagents.agents import CodeAgent, ToolCallingAgent

In [20]:
model = OpenAIServerModel(
    # model_id="qwen2.5-coder:latest",
    # model_id="ishumilin/deepseek-r1-coder-tools:14b",
    # model_id="qwen2.5-coder:14b",
    model_id="ishumilin/deepseek-r1-coder-tools:14b",
    api_base="http://localhost:11434/v1",
    api_key="ollama",
    temperature=0.6,
    max_tokens=8192,  # 设置生成的最大token数
)

In [3]:
@tool
def get_server_status(server_name: str) -> str:
    """
    Get the running status of a specified server from the Zabbix server

    Args:
        server_name: the server name
    """
    return f"""服务器 {server_name} 的运行状态如下：

      1. 基础状态：正常运行中
      2. 运行时间：15天6小时
      3. 系统负载：
         - CPU使用率：94%
         - 内存使用：6%
         - 磁盘使用：7%

      4. 关键服务：
         - Web服务：运行正常
         - 数据库：运行正常
         - 缓存服务：超出负载

      5. 网络状态：
         - 响应时间：1200ms
         - 连接数：6
         
      最后检查时间：2024-03-20 15:30:45"""

In [4]:
@tool
def get_status(server_names: list[str]) -> str:
    """
    Get the running status of specified servers from the Zabbix server

    Args:
        server_names: list of server names to check
    """
    status_template = """服务器 {name} 的运行状态如下：

      1. 基础状态：{basic_status}
      2. 运行时间：{uptime}
      3. 系统负载：
         - CPU使用率：{cpu_usage}
         - 内存使用：{mem_usage}
         - 磁盘使用：{disk_usage}

      4. 关键服务：
         - Web服务：{web_status}
         - 数据库：{db_status}
         - 缓存服务：{cache_status}

      5. 网络状态：
         - 响应时间：{response_time}
         - 连接数：{connections}
         
      最后检查时间：2024-03-20 15:30:45
      """
    
    # 模拟不同服务器的状态
    server_statuses = {
        "1号服务器": {
            "basic_status": "正常运行中",
            "uptime": "15天6小时",
            "cpu_usage": "94%",
            "mem_usage": "6%",
            "disk_usage": "7%",
            "web_status": "运行正常",
            "db_status": "运行正常",
            "cache_status": "超出负载",
            "response_time": "1200ms",
            "connections": "6"
        },
        "2号服务器": {
            "basic_status": "正常运行中",
            "uptime": "23天4小时",
            "cpu_usage": "45%",
            "mem_usage": "72%",
            "disk_usage": "65%",
            "web_status": "运行正常",
            "db_status": "运行正常",
            "cache_status": "运行正常",
            "response_time": "200ms",
            "connections": "12"
        },
        "3号服务器": {
            "basic_status": "正常运行中",
            "uptime": "7天12小时",
            "cpu_usage": "88%",
            "mem_usage": "85%",
            "disk_usage": "23%",
            "web_status": "响应延迟",
            "db_status": "运行正常",
            "cache_status": "运行正常",
            "response_time": "800ms",
            "connections": "15"
        }
    }

    result = []
    for server_name in server_names:
        if server_name in server_statuses:
            status = server_statuses[server_name]
            result.append(status_template.format(
                name=server_name,
                **status
            ))
        else:
            result.append(f"未找到服务器 {server_name} 的状态信息")
    
    return "\n\n" + "\n\n".join(result)

In [5]:
@tool
def lookup_manual(fault_phenomenon: str) -> str:
    """
    Look up relevant content in the operation manual based on the fault phenomenon description.

    Args:
        fault_phenomenon: the fault phenomenon description
    """
    
    return f"""收到故障现象：{fault_phenomenon}

根据运维手册，对于该故障现象的处理建议如下：

1. 性能指标检查：
   - CPU使用率 (警戒值>90%)
   - 内存使用率 (警戒值>80%)
   - 磁盘使用率 (警戒值>85%)
   - 网络响应时间 (警戒值>800ms)
   - 连接数监控 (参考值：单服务器建议<20)

2. 关键服务检查：
   - Web服务状态和配置
   - 数据库连接池设置
   - 缓存服务负载和配置
   - 应用服务器日志分析

3. 常见解决方案：
   - 高CPU使用率：检查是否存在资源密集型进程，必要时进行限制
   - 缓存服务超载：调整缓存配置，增加容量或优化策略
   - 响应延迟：检查网络带宽、连接池配置、数据库索引
   - 服务异常：重启相关服务，回滚最近配置变更

4. 应急处理流程：
   - 执行操作前备份关键数据和配置
   - 记录所有处理步骤和结果
   - 如超出处理能力及时上报
   - 保留故障现场信息供后续分析

5. 预防措施：
   - 配置监控告警阈值
   - 定期进行性能优化
   - 建立服务器性能基线
   - 保持系统包更新

注意：如果服务器负载持续异常，建议进行详细的性能分析和优化。"""

In [6]:
@tool
def lookup_history(fault_phenomenon: str) -> str:
    """
    Look up similar fault cases from historical records based on the fault phenomenon description.

    Args:
        fault_phenomenon: the fault phenomenon description
    
    Returns:
        str: Historical fault resolution information or a message if no relevant cases found
    """

    his_rec1 = f"""收到故障现象：{fault_phenomenon}
根据历史记录，找到以下相似故障案例：

1. 最近发生时间：2024-02-15
   解决方案：重启服务并更新了配置文件

2. 最近发生时间：2024-01-20
   解决方案：升级安全补丁至2.1.3版本

3. 最近发生时间：2023-12-05
   解决方案：清理了系统日志和临时文件，释放磁盘空间

如果以上方案都无法解决问题，请上报至高级运维团队处理。"""

    his_rec2 = """收到故障现象：{fault_phenomenon}
根据历史记录，没有找到相似故障案例：
"""

    his_rec3 = f"""收到故障现象：{fault_phenomenon}
根据历史记录，找到以下相似故障案例：

1. 最近发生时间：2024-02-15
   故障现象：Web应用响应时间超过1秒
   原因分析：缓存服务配置不当导致缓存命中率低，增加了数据库负载
   解决方案：优化Redis缓存配置，增加缓存容量并调整过期策略

2. 最近发生时间：2024-01-20
   故障现象：系统响应缓慢，CPU使用率持续高位
   原因分析：应用程序存在内存泄漏，导致垃圾回收频繁
   解决方案：升级应用版本至2.1.3，修复了内存管理问题

3. 最近发生时间：2023-12-05
   故障现象：Web服务间歇性响应慢
   原因分析：数据库连接池配置过小，并发请求排队等待
   解决方案：调整数据库连接池参数，增加最大连接数

如果以上方案都无法解决问题，请上报至高级运维团队处理。"""

    his_rec4     = f"""收到故障现象：{fault_phenomenon}
根据历史记录，没有找到相似故障案例。
建议参考运维手册进行标准故障处理流程。
"""
    return his_rec3



In [21]:
agent = CodeAgent(tools=[get_status, lookup_manual, lookup_history], 
model=model,
verbosity_level=2)

In [ ]:
# print("CodeAgent:", agent.run("1号服务器出现web应用反应缓慢的现象，检查原因"))
instruct1 = f"""1号服务器和3号服务器出现web应用反应缓慢的现象
按照下面的步骤检查原因：
- 获取服务器的运行状态，解析状态；
- 根据故障现象从操作手册中查找相关的内容；
- 根据故障现象从历史记录中查找相关的内容；
- 综合以上信息，分析可能的原因
"""

instruct2 = f"""1号服务器 和 3号服务器 出现web应用反应缓慢的现象
按照下面的步骤检查原因：
- 获取服务器的运行状态，解析状态；
- 根据故障现象从操作手册中查找相关的内容；
- 根据故障现象从历史记录中查找相关的内容；
- 综合以上信息，分析可能的原因,并给出建议；
"""


instruct3 = f"""The 1号服务器 and 2号服务器 are experiencing a phenomenon of slow web application response.
Check the cause according to the following steps:
- Obtain and parse the server's running status;
- Look up relevant information from the operation manual based on the fault phenomenon;
- Look up relevant information from the history records based on the fault phenomenon;
- Analyze the possible cause based on the above information.

"""

instruct4 = f"""1号服务器出现web应用反应缓慢的现象
根据运维手册、历史记录和服务器运行状态分析可能的原因
"""
instruct5 = f"""1号服务器出现web应用反应缓慢的现象，分析可能的原因，给出建议
"""

print("CodeAgent:", agent.run(instruct2))

In [ ]:
agent.replay(detailed=True) 